
# ARIMA: Autoregressive Integrated Moving Average

## By Team Fourier
CS189 Project T Final

This notebook will provide an introduction into ARIMA, short for autoregressive integrated moving average. ARIMA models aim to model time series data such as the weather, stock prices, or product sales over a period of time. The focus will be on weather data which has a seasonal aspect to it. The notebook will move into SARIMAX, short for seasonal autoregressive integrated moving average, a model that allows for seasonal periodic behavior. The notebook will rely heavily on the statsmodels Python package.

## Learning Objectives

1. Provide some insight into the power of time series analysis and moving average models as opposed to traditional function fitting techniques such as linear regression.

2. Provide a brief introduction to techniques in order to determine important characteristics of your data (stationarity, seasonality) that will influence decisions when choosing a model.

3. Give an introduction into statsmodels ARIMA and SARIMAX capabilities. In real applications, these functions are what you would use to implement an ARIMA model yourself.

4. Provide some examples of visualization, an important skill for data scientists.

5. A brief example of using grid search in order to choose optimal hyperparameters.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from statsmodels.tsa.statespace.sarimax import SARIMAX

import datetime as dt

In [ ]:
data = pd.read_csv('data/TSLA.csv')
time = np.array(data['Date'])
price = np.array(data['Close'])

plt.title('Tesla Stock Price')
plt.plot(time, price)
plt.xticks(time[::50])
plt.xlabel('Days since November 19, 2019')
plt.ylabel('Price')
plt.show()

For example, the graph above depicts the stock price of TSLA over time. The x-axis being time and the y-axis being f(t), the price at that point in time.

Clearly a basic linear model wouldn't be able to model such a function of time due to its non-linearity. However, we have previously learned ways to fit non-linear functions with linear regression using feature lifting. Maybe lifting some polynomial features might help?

## Trying Linear Regression

Linear features, linear regression

In [ ]:
time_steps = np.arange(0, time.shape[0], 1).astype(np.int).reshape((time.shape[0], 1))

#TODO: run a least squares linear regression in the format Ax=b, A = time_steps, B = price
#place the result in w
#Hint: try np.linalg.lstsq
w = 0

plt.plot(time_steps, w * time_steps, label='linear fit')
plt.plot(time_steps, price, label = 'original data')
plt.legend()
plt.title('Tesla Stock Price with linear fit')
plt.xlabel('Days since November 19, 2019')
plt.ylabel('Price')
plt.show()


Polynomial features, linear regression

In [ ]:
def feature_lift(X, d=2):
    """
    This function computes second order variables
    for polynomial regression.
    Input:
    X: Independent variables.
    Output:
    A data matrix composed of both first and second order terms.
    """
    
    X = np.array(X)
    X_data = []
    
    for deg in range(1, d+1):
        X_data.append(np.power(X, deg))
        
    return np.hstack(X_data)

second_order_time_steps = feature_lift(time_steps, d=2)
w_2, err_2, _, _ = np.linalg.lstsq(second_order_time_steps[:210], price[:210])

third_order_time_steps = feature_lift(time_steps, d=3)
w_3, err_3, _, _ = np.linalg.lstsq(third_order_time_steps[:220], price[:220])

fifty_order_time_steps = feature_lift(time_steps, d=50)
w_50, err_50, _, _ = np.linalg.lstsq(fifty_order_time_steps[:220], price[:220])

plt.plot(time_steps, second_order_time_steps @ w_2.T, label='deg 2 fit')
plt.plot(time_steps, third_order_time_steps @ w_3.T, label='deg 3 fit')
plt.plot(time_steps, fifty_order_time_steps @ w_50.T, label='deg 50 fit')
plt.plot(time_steps, price, label = 'original data')

plt.xlim(right=250)
plt.xlabel('Days since November 19, 2019')
plt.ylabel('Price')
plt.legend()
plt.title('Tesla stock price with linear regression on feature lift')
plt.show()

print('Training error with degree 2 features: %s' % str(err_2))
print('Training error with degree 3 features: %s' % str(err_3))
print('Training error with degree 50 features: %s' % str(err_50))

Basically, any attempt to fit some sort of function using linear regression and feature lifting will be met with failure. Why? This is due to the nature of time series data. The problem that linear regressions try to solve is fitting some function to the data in its entirety. However, time series data, such as stock prices don't operate under the same assumptions. Does the stock price 1 year ago directly affect the stock price tomorrow? The answer is, not really. This understanding that only the previous n' << n observations are truly important to our model spawns this set of models that utilize the moving average, the first MA model, then the ARMA, and ARIMA models.

# Formulation of the ARIMA Model

The mathematical model behind ARIMA is:

\begin{align*}
ARIMA(p, d, q) = (1 - \phi_{1}L - \phi_{2}L^{2} - \phi_{3}L^{3}... - \phi_{p}L^{p}) (1 - L)^{d} y_{t} = c + (1 + \theta_{1} L + \theta_{2} L^{2}... + \theta_{q} L^{q}) \epsilon_{t}
\end{align*}

The L is the lag operator, $L^d y_{t} = y_{t - d}$.

The first group of terms, containing $\phi$'s, are the auto-regressive terms. Auto-regressive refers to the contribution of previous observations on the current observation. The parameter $p$ is the number of previous observations looked at. The $\phi$'s are their individual weighting. Intuitively, an observation 1 time step ago usually holds more weight than an observation 10 time steps ago.

The second term, $(1 - L)^{d}$, is the integrated term. This is the term that implements the idea of differencing, which is sometimes necessary to find a deeper relationship in the data. The time series function may not be able to model the observation themselves, but may be able to model the difference between successive observations.

The third and final term, containing $\theta$'s, represent the moving average portion of the model. This is the contribution of the previous $q$ terms, specifically their residuals to the residual at time t. The residuals, $\epsilon_{t}$, are standard normal distribution deviations of each observation from the predicted observation predicted by the model. These residuals are what we try to minimize when fitting parameters to the training data.

The constant c quantifies the drift of the model.

# Breaking Down the ARIMA Model

## Auto-Regressive (AR)

One of the fundamental building blocks of the ARIMA model is the AR portion of the model. The equation for a simple AR model is:

\begin{align*}
AR(p) = y_{t} = c + (\phi_{1}L + \phi_{2}L^{2} + \phi_{3}L^{3}... + \phi_{p}L^{p})y_{t} + \epsilon_{t}
\end{align*}

This equation can already begin to model functions under the assumption that current observations are simply derived from a linear combination of the p previous observations.



For the rest of this notebook, we'll be working with weather data instead of stock data.

In [ ]:
#load dataset
weather_data = pd.read_csv('data/weather_data.csv', parse_dates=['datetime_utc'], index_col='datetime_utc')
weather_data = weather_data.rename(index=str, columns={' _tempm': 'temperature'})

#interpolate null values
weather_data.ffill(inplace=True)
weather_data.index = pd.to_datetime(weather_data.index)

#remove outliers
weather_data = weather_data[weather_data.temperature < 50]
weather_temp = weather_data['temperature']

weather_temp.plot()
plt.xlabel('time')
plt.ylabel('temp in deg C')
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Historical weather data')
plt.show()

print(weather_temp.head())
print('Null values: %s' % str(weather_temp.isnull().sum()))

The first thing we do with our data is clean it, interpolating null values and making sure the samples are evenly spaced. Next, there are some things we can check about the data itself to ensure it is suitable for an ARIMA model.

In [ ]:
training_data = weather_temp['2000':'2014'].resample('M').mean().fillna(method='pad')
testing_data = weather_temp['2015':'2017'].resample('M').mean().fillna(method='pad')
training_data.plot()
testing_data.plot()
plt.title('Training and Testing Data Split')
plt.show()

In the next cell we will plot the autocorrelation of both the original data and the differences between consecutive data points. There is also a provided function ``plot_rolling_mean_std``. Try to find a period length that creates a constant rolling mean. What does this say about the periodicity of the data?

In [ ]:
# check rolling mean and rolling standard deviation
def plot_rolling_mean_std(ts, period):
    rolling_mean = ts.rolling(period).mean()
    rolling_std = ts.rolling(period).std()

    plt.plot(ts, label='Actual Mean')
    plt.plot(rolling_mean, label='Rolling Mean')
    plt.plot(rolling_std, label = 'Rolling Std')
    plt.xlabel("Date")
    plt.ylabel("Mean Temperature")
    plt.title('Rolling Mean & Rolling Standard Deviation')
    plt.legend()
    plt.show()

#TODO: change this period parameter until the resulting rolling mean is 0 (Hint: What do you think the period of this data is?)
plot_rolling_mean_std(training_data, period = 2)
    
plt.plot(training_data)
plot_acf(training_data)
plt.show()

plt.plot(training_data.diff())
plot_acf(training_data.diff().values)
plt.show()


We see from the rolling mean and rolling standard deviation plot that the data has a constant mean and standard deviation on a 12 month period. This periodicity implies that our data is seasonal, meaning a standard ARIMA model will have trouble fitting it, but instead we will need to use the seasonal aspect of SARIMA to fit the data.

First, we will try to see what kind of model we retrieve from an AR model.

In [ ]:
def process_result(res, model='arima'):
    print(res.summary())
    
    plt.plot(res.resid)
    plt.title('Training Residuals')
    plt.xlabel('Months')
    plt.ylabel('Temperature in C')
    plt.show()

    print('Mean squared training error: %s' % str(np.square(res.resid).mean()))

    plt.plot(res.predict(), label='Prediction')
    plt.plot(training_data.values, label='Real values')
    plt.legend()
    plt.title('Model fit on training data')
    plt.xlabel('Months')
    plt.ylabel('Temperature in C')
    plt.show()
    
    if model == 'arima':
        fc, se, conf = res.forecast(testing_data.shape[0], alpha=0.05)
    elif model == 'sarimax':
        fc = res.forecast(testing_data.shape[0], alpha=0.05)
        
    fc_series = pd.Series(fc, index=testing_data.index)
    plt.plot(fc_series, label='Prediction')
    plt.plot(testing_data, label='Real values')
    plt.legend()
    plt.title('Model prediction on test data')
    plt.xlabel('Datetime')
    plt.ylabel('Temperature in C')
    plt.show()
    
    #TODO: Calculate the mean squared testing error
    squared_testing_error = np.square(fc - testing_data).mean()
    print('Mean squared testing error: %s' % str(squared_testing_error))

In [ ]:
#TODO: use ARIMA(data, order).fit() to generate an AR model fit
#data = training_data.values
#order = (p, 0, 0)
#Hint: try p > 1
#https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_model.ARIMA.html
res = None
process_result(res)


### Write down your observations.

How does the model perform on the training data?

How does the model perform on the forecasting of future data?

Why does the AR model not fit this seasonal data well?

## Moving Average (MA)

Another fundamental building block of the ARIMA model is the MA portion of the model. The equation for a simple MA model is:

\begin{align*}
MA(q) = y_{t} = c + \epsilon_{t} + \theta_{1}\epsilon_{t-1} + \theta_{2}\epsilon_{t-2} + ... + \theta_{q}\epsilon_{t-q}
\end{align*}

This equation models the assumption that the residual, the deviation from the predicted point, of the current observation is a linear combination of the previous q residuals.

Add the moving average section of the ARIMA model, and model the weather data using an ARMA model.

In [ ]:
#TODO: use ARIMA(data, order).fit() to generate an ARMA model fit
#data = training_data.values
#order = (p, 0, q)
#Hint: try p, q > 1
#https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_model.ARIMA.html
res = None
process_result(res)

### Write down your observations.

How does the model perform on the training data?

How does the model perform on the forecasting of future data?

Does the ARMA model fit the seasonal data better than the AR model?

# Seasonality in ARIMA: SARIMA

As we noted in the weather data, we see a seasonal pattern, or period with fixed period length. The traditional ARIMA model is not fully suited to model this behavior, but this is where SARIMA comes into play. SARIMA, season auto-regressive integrated moving average, has a slightly different formulation from ARIMA:

\begin{align*}
SARIMA(p, d, q) (P, D, Q)_{m} = (1 - \phi_{1}L - \phi_{2}L^{2} - \phi_{3}L^{3}... - \phi_{p}L^{p}) (1 - \Phi_{1}L^{m} - \Phi_{2}L^{2m} - \Phi_{3}L^{3m}... - \Phi_{P}L^{Pm}) (1 - L)^{d} (1 - L)^{Dm} y_{t} = c + (1 + \theta_{1} L + \theta_{2} L^{2}... + \theta_{q} L^{q}) (1 + \Theta_{1} L^{m} + \Theta_{2} L^{2m}... + \Theta_{Q} L^{Qm}) \epsilon_{t}
\end{align*}

The seasonal terms are terms that operate much like the original ARIMA terms, but the lag operators are all raised to the power of $m$, in the case of the annual cycle of weather data, $m=12$, meaning the model takes into account observations from 12, 24, 36.... months ago.

Implement a SARIMA model on the weather data.

In [ ]:
#TODO: use SARIMAX(data, order, seasonal_order).fit() to generate a SARIMAX model fit
#data = training_data.values
#order = (p, 0, q)
#seasonal_order = (P, 0, Q, S)
#Hint: try p, q > 1 and P, Q > 0
#https://www.statsmodels.org/stable/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html
res = None
process_result(res, 'sarimax')

### Write down your observations.

How does the model perform on the training data?

How does the model perform on the forecasting of future data?

Does the SARIMAX model fit the seasonal data better than the ARIMA model?

# Hyperparameters

You may have noticed that sometimes the performance is better than other times when using different values for p, d, q, P, D, and Q. These values are called hyperparameters, hand selected values by the engineer creating the model that greatly influence the effectiveness of the model. There are several ways to set these hyperparameters, as discussed earlier in this course. You can employ techniques such as k-fold cross validation and grid searches, as well as use domain knowledge to make educated guesses on what we want the model to learn and what the underlying model in reality is. For example, someone with strong knowledge of the weather can make a better educated guess about how many days one should look back in the past to predict today's weather.

First, make a guess based on what you think influences the weather. What are good values for p, d, q, P, D, and Q? 

Hint: Don't use d or D, differencing leads to a very bad result in this dataset.

Run a SARIMAX model with your guess here:

In [ ]:
#TODO run a SARIMAX model with your guess
res = None
process_result(res, 'sarimax')

Now, let's try using a grid search to find the best parameters.

In [ ]:
def get_test_mse(res):
    #TODO calculate test mse
    #refer to function process_result in this notebook on how to use res.forecast()
    fc = None
    squared_testing_error = np.square(fc - testing_data).mean()
    return squared_testing_error

p_range = list(range(4))
q_range = list(range(4))
P_range = list(range(4))
Q_range = list(range(4))

best_params = (-1, -1, -1, -1)
best_test_mse = 10000000

#todo use a grid search to find the best parameters
for p in p_range:
    for q in q_range:
        for P in P_range:
            for Q in Q_range:
                try:
                    res = SARIMAX(training_data.values, order=(p, 0, q), seasonal_order=(P, 0, Q, 12), maxiter = 5).fit()
                    test_mse = get_test_mse(res)
                    if test_mse < best_test_mse:
                        best_params = (p, q, P, Q)
                        best_test_mse = test_mse
                    print('Tested params:', p, q, P, Q)
                except:
                    pass
                
                
print('Best parameters: ', best_params)
res = SARIMAX(training_data.values, order=(best_params[0], 0, best_params[1]), seasonal_order=(best_params[2], 0, best_params[3], 12)).fit()
process_result(res, 'sarimax')

That probably took a while, was it worth it?

## Observations

How did the grid search perform compared to your guess?